In [15]:
# Import key librarys

import pandas as pd
import numpy as np

import plotly.express as px

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

In [16]:
# Define level
level = 'Advanced_Higher'
level_label = level.replace('_', ' ')

# Import data
ind_df = pd.read_excel("./data/attainment-statistics-(august)-2022-provisional-centre-independent.ods", 
                       engine = "odf", sheet_name = level, skiprows = 2)

state_df = pd.read_excel("./data/attainment-statistics-(august)-2022-provisional-centre-education-authority.ods", 
                        engine = "odf", sheet_name = level, skiprows = 2)

In [17]:
def tidy_up_1(df):
    
    # Remove spaces and replace with '_'
    df.columns = df.columns.str.replace(' ','_')

    # Replace weird values with NaN
    df = df.replace('[z]', np.nan)
    df = df.replace('[c]', np.nan)
    df = df.replace('[low]', np.nan)

    # Remove last row
    df = df.drop(df.index[-1])

    # Remove columns with Percentage in the heading
    df = df[df.columns.drop(list(df.filter(regex = 'Percentage')))]
    
    return df
    
ind_df = tidy_up_1(ind_df)
state_df = tidy_up_1(state_df)

In [18]:
def tidy_up_2(df):

    # Create a list of years
    years = ['2022', '2021', '2020', '2019', '2018']

    # Sort out column titles
    for year in years:
        df = df.rename({'Grade_A_Count_' + year: 'As_' + year}, axis=1)
        df = df.rename({'Grades_A-B_Count_' + year: 'A-Bs_' + year}, axis=1)
        df = df.rename({'Grades_A-C_Count_' + year: 'A-Cs_' + year}, axis=1)
        df = df.rename({'Grades_A-D_Count_' + year: 'A-Ds_' + year}, axis=1)
        df = df.rename({'No_Award_Count_' + year: 'NAs_' + year}, axis=1)

    # Calculate number of Bs, Cs and Ds
    for year in years:
        df['Bs_' + year] = df['A-Bs_' + year] - df['As_' + year]
        df['Cs_' + year] = df['A-Cs_' + year] - df['A-Bs_' + year]
        df['Ds_' + year] = df['A-Ds_' + year] - df['A-Cs_' + year]

    # Remove original columns
    for year in years:
        df.drop('A-Bs_' + year, axis=1, inplace=True)
        df.drop('A-Cs_' + year, axis=1, inplace=True)
        df.drop('A-Ds_' + year, axis=1, inplace=True)
        
    return df
        
ind_df = tidy_up_2(ind_df)
state_df = tidy_up_2(state_df)

ind_df.head()

,Subject,As_2022,NAs_2022,Entries_2022,As_2021,NAs_2021,Entries_2021,As_2020,NAs_2020,Entries_2020,...,Ds_2021,Bs_2020,Cs_2020,Ds_2020,Bs_2019,Cs_2019,Ds_2019,Bs_2018,Cs_2018,Ds_2018
0,Accounting,10.0,0.0,15.0,20.0,0.0,25.0,15.0,0.0,20.0,...,0.0,0.0,5.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0
1,Art and Design (Design),40.0,0.0,60.0,35.0,0.0,45.0,35.0,0.0,45.0,...,0.0,5.0,5.0,0.0,15.0,5.0,0.0,15.0,10.0,0.0
2,Art and Design (Expressive),65.0,0.0,95.0,65.0,0.0,85.0,75.0,0.0,95.0,...,0.0,15.0,5.0,0.0,20.0,5.0,0.0,10.0,10.0,0.0
3,Biology,200.0,15.0,465.0,320.0,5.0,485.0,250.0,NaN,430.0,...,15.0,115.0,55.0,10.0,95.0,65.0,30.0,110.0,75.0,20.0
4,Business Management,90.0,10.0,240.0,110.0,NaN,165.0,120.0,0.0,180.0,...,5.0,35.0,25.0,0.0,30.0,30.0,5.0,40.0,25.0,10.0


In [19]:
def as_per(df):

    # Create a list of years
    years = ['2022', '2021', '2020', '2019', '2018']

    # Calculate perc of As
    for year in years:
        
        df['perAs_' + year] = round(df['As_' + year] / df['Entries_' + year], 2)
        
    return df

ind_df = as_per(ind_df)
state_df = as_per(state_df)
        
ind_df.tail()

,Subject,As_2022,NAs_2022,Entries_2022,As_2021,NAs_2021,Entries_2021,As_2020,NAs_2020,Entries_2020,...,Cs_2019,Ds_2019,Bs_2018,Cs_2018,Ds_2018,perAs_2022,perAs_2021,perAs_2020,perAs_2019,perAs_2018
28,Physical Education,55.0,0.0,125.0,80.0,0.0,95.0,50.0,0.0,65.0,...,5.0,5.0,15.0,10.0,0.0,0.44,0.84,0.77,0.40,0.40
29,Physics,160.0,5.0,290.0,215.0,NaN,295.0,190.0,NaN,280.0,...,30.0,10.0,75.0,35.0,5.0,0.55,0.73,0.68,0.51,0.53
30,"Religious, Moral and Philosophical Studies",50.0,NaN,100.0,80.0,0.0,95.0,60.0,0.0,95.0,...,20.0,0.0,15.0,10.0,5.0,0.50,0.84,0.63,0.53,0.42
31,Spanish,55.0,NaN,90.0,65.0,NaN,85.0,65.0,0.0,80.0,...,10.0,5.0,20.0,20.0,5.0,0.61,0.76,0.81,0.53,0.47
32,Statistics,45.0,5.0,95.0,95.0,NaN,125.0,60.0,NaN,90.0,...,15.0,5.0,15.0,20.0,5.0,0.47,0.76,0.67,0.41,0.53


In [20]:
# Convert into long format

def tolong(df):
    ldf = pd.melt(df, id_vars = ['Subject'], var_name = 'abc', value_name ='Count')
    ldf['Grades'] = ldf['abc'].apply(lambda x: (x.rsplit('_', 1))[0])
    ldf['Year'] = ldf.pop('abc').apply(lambda x: (x.split('_'))[-1])
    
    return ldf
    
ind_ldf = tolong(ind_df)
state_ldf = tolong(state_df)

ind_ldf['School'] = 'Ind'
state_ldf['School'] = 'State'

ind_ldf.tail()

,Subject,Count,Grades,Year,School
1150,Physical Education,0.40,perAs,2018,Ind
1151,Physics,0.53,perAs,2018,Ind
1152,"Religious, Moral and Philosophical Studies",0.42,perAs,2018,Ind
1153,Spanish,0.47,perAs,2018,Ind
1154,Statistics,0.53,perAs,2018,Ind


In [21]:
# Merge together two df
#dfc = pd.merge(state_ldf, ind_ldf, on = ['Subject', 'Grades', 'Year', 'Count'])
dfc = pd.concat([state_ldf, ind_ldf])

dfc.head()

,Subject,Count,Grades,Year,School
0,Accounting,15.0,As,2022,State
1,Art and Design (Design),185.0,As,2022,State
2,Art and Design (Expressive),410.0,As,2022,State
3,Biology,570.0,As,2022,State
4,Business Management,110.0,As,2022,State


In [22]:
dfc = dfc[dfc.Grades == 'perAs']
dfc.dropna(inplace=True)

dfc.tail()

,Subject,Count,Grades,Year,School
1150,Physical Education,0.40,perAs,2018,Ind
1151,Physics,0.53,perAs,2018,Ind
1152,"Religious, Moral and Philosophical Studies",0.42,perAs,2018,Ind
1153,Spanish,0.47,perAs,2018,Ind
1154,Statistics,0.53,perAs,2018,Ind


In [23]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 1050 to 1154
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Subject  296 non-null    object 
 1   Count    296 non-null    float64
 2   Grades   296 non-null    object 
 3   Year     296 non-null    object 
 4   School   296 non-null    object 
dtypes: float64(1), object(4)
memory usage: 13.9+ KB


In [30]:
# Plot graph

# Create a list of subjects in alphabetical order
subjects = sorted(dfc['Subject'].unique())
years = sorted(dfc['Year'].unique())

fig = px.line(dfc, x = "Year", y = "Count", 
                color = "Subject",
                facet_row = 'School',
                width = 800,
                height = 1150,
                category_orders = {'Subject' : subjects, 'Year' : years},
                labels = {"Count": "Percentage"},
                title = 'Percentage of As per entry per subject from 2018 - 2022 at ' + level_label
                )

fig.update_traces(mode = "markers+lines")

fig.update_yaxes(matches=None, rangemode = "tozero")

filename = 'As_percentage_line' + '_' + level

fig.write_html('./graphs/' + filename  + '.html')

fig.show()

In [32]:
# Setup Chart Studio
username = '###' # your username
api_key = '###' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# Upload to Chart Studio
#py.plot(fig, filename = filename, auto_open=True)